In [1]:
import sqlite3 as sqlt
import pandas as pd
from datetime import datetime

In [18]:
querie = """select * from exports where "Cod. Mun" in {} and 
"NCM" like "6402%";"""
grendene_cidades = "(4307906, 4304804,  2312908, 2931350, 2304202, 2304400)"
alpargatas_cidades = "(2604007, 2504009, 3143302, 3530607)"


In [19]:
con = sqlt.connect("big_data2.db")
cursor = con.cursor()

In [12]:
def data_clean_db(dataframe):
    dict_nomes_colunas = {"month":"data","US$ de P1":"financeiro","Desc. Mun":"municipio",
                          "Kg Liq de P1":"peso","Qtd de P1":"quantidade"}
    dataframe.rename(columns=dict_nomes_colunas,inplace=True)
    dataframe.drop(axis=1,columns=["Desc. NCM"],inplace=True)
    dataframe.data = dataframe.data.apply(
        lambda data: datetime.strptime(data.split()[0], "%Y-%m-%d"))
    dataframe.set_index("index", inplace=True)
    dataframe["Cod. Mun"] = dataframe["Cod. Mun"].astype("str")
    dataframe.NCM = dataframe.NCM.astype("str")
    dataframe.financeiro = dataframe.financeiro.astype("int")
    dataframe.peso = dataframe.peso.astype("int") 
    
    return dataframe

def data_clean_xlsx(dataframe):
    dataframe.reset_index(inplace=True)
    grendene_exports_of = dataframe[["index","Volume exportado Grendene"]]
    grendene_exports_of.rename(columns={"index":"data","Volume exportado Grendene":"volume"},inplace=True)

    alpargatas_exports_of = dataframe[["Unnamed: 2","Volume exportado Alpargatas"]]
    alpargatas_exports_of.rename(columns={"Unnamed: 2":"data","Volume exportado Alpargatas":"volume"},inplace=True)
    grendene_exports_of.data = grendene_exports_of.data.apply(lambda data:data.date())
    alpargatas_exports_of.data = alpargatas_exports_of.data.apply(lambda data:data.date())
    
    alpargatas_exports_of.dropna(inplace=True)
    grendene_exports_of.volume = grendene_exports_of.volume.astype("int")
    alpargatas_exports_of.volume = alpargatas_exports_of.volume.astype("int")

    return (grendene_exports_of, alpargatas_exports_of)

In [ ]:
grendene_db = pd.read_sql_query(querie.format(grendene_cidades), con)
alpargatas_db = pd.read_sql_query(querie.format(alpargatas_cidades), con)

grendene_db = data_clean_db(grendene_db)
alpargatas_db = data_clean_db(alpargatas_db)

In [14]:
grendene_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1038 entries, 571 to 35701
Data columns (total 7 columns):
Cod. Mun      1038 non-null object
municipio     1038 non-null object
NCM           1038 non-null object
financeiro    1038 non-null int64
peso          1038 non-null int64
quantidade    1038 non-null int64
data          1038 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 64.9+ KB


In [15]:
exportacoes_oficiais = pd.read_excel("Exportação calçados.xlsx")
grendene_exports_of, alpargatas_exports_of = data_clean_xsls(exportacoes_oficiais)

/home/rafa/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/home/rafa/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/rafa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
for index, linha in grendene_exports_of.iterrows():
    soma = grendene_db[grendene_db.data==linha["data"]].financeiro.sum()
    string = "data: {}\nvalor movimentado oficial: {}\nvalor movimentado no db:   {}\n\n"
    print(string.format(linha["data"], linha["volume"],soma))

data: 2009-03-01
valor movimentado oficial: 15722000
valor movimentado no db:   11944829


data: 2009-06-01
valor movimentado oficial: 8141000
valor movimentado no db:   7911437


data: 2009-09-01
valor movimentado oficial: 7724000
valor movimentado no db:   7493393


data: 2009-12-01
valor movimentado oficial: 16698000
valor movimentado no db:   13119399


data: 2010-03-01
valor movimentado oficial: 21201000
valor movimentado no db:   19620984


data: 2010-06-01
valor movimentado oficial: 10173000
valor movimentado no db:   10515877


data: 2010-09-01
valor movimentado oficial: 9213000
valor movimentado no db:   17392900


data: 2010-12-01
valor movimentado oficial: 14046000
valor movimentado no db:   23130423


data: 2011-03-01
valor movimentado oficial: 12832000
valor movimentado no db:   11603149


data: 2011-06-01
valor movimentado oficial: 7481000
valor movimentado no db:   6839422


data: 2011-09-01
valor movimentado oficial: 9313000
valor movimentado no db:   8500638


data: 20

In [17]:
for index, linha in alpargatas_exports_of.iterrows():
    soma = alpargatas_db[alpargatas_db.data==linha["data"]].financeiro.sum()
    string = "data: {}\nvalor movimentado oficial: {}\nvalor movimentado no db:   {}\n\n"
    print(string.format(linha["data"], linha["volume"],soma))

data: 2010-03-01
valor movimentado oficial: 6500000
valor movimentado no db:   6198785


data: 2010-06-01
valor movimentado oficial: 7500000
valor movimentado no db:   4336305


data: 2010-09-01
valor movimentado oficial: 6100000
valor movimentado no db:   5720937


data: 2010-12-01
valor movimentado oficial: 6100000
valor movimentado no db:   7886964


data: 2011-03-01
valor movimentado oficial: 7600000
valor movimentado no db:   5540689


data: 2011-06-01
valor movimentado oficial: 8300000
valor movimentado no db:   7246531


data: 2011-09-01
valor movimentado oficial: 6100000
valor movimentado no db:   6769557


data: 2011-12-01
valor movimentado oficial: 6100000
valor movimentado no db:   7740834


data: 2012-03-01
valor movimentado oficial: 7692000
valor movimentado no db:   9975289


data: 2012-06-01
valor movimentado oficial: 10484000
valor movimentado no db:   9916039


data: 2012-09-01
valor movimentado oficial: 5797000
valor movimentado no db:   12162386


data: 2012-12-01
va